In [13]:
import torch
import pandas as pd
import pyaging as pya
import json
import os

In [14]:
%%writefile dunedinpace.r

options(repos = c(CRAN = "https://cloud.r-project.org/"))

install.packages("devtools")

devtools::install_github("danbelsky/DunedinPACE", build_vignettes = FALSE)

library(DunedinPACE)
library(jsonlite)

PACE_list = list(c(
    mPACE_Models$model_names, 
    mPACE_Models$gold_standard_probes, 
    mPACE_Models$model_weights, 
    mPACE_Models$model_intercept,
    mPACE_Models$model_means,
    mPACE_Models$model_probes,
    mPACE_Models$gold_standard_means    
))

write_json(PACE_list, "DunedinPACE.json")

Writing dunedinpace.r


In [6]:
os.system("Rscript dunedinpace.r")
os.system("rm dunedinpace.r")

0

In [16]:
with open('DunedinPACE.json', 'r') as f:
    PACE_list = json.load(f)[0]

In [21]:
features = PACE_list['DunedinPACE.4']

weights = torch.tensor(PACE_list['DunedinPACE.1']).unsqueeze(0)
intercept = torch.tensor([PACE_list['DunedinPACE.2'][0]])

In [30]:
PACE_list.keys()

dict_keys(['1', 'DunedinPACE', 'DunedinPACE.1', 'DunedinPACE.2', 'DunedinPACE.3', 'DunedinPACE.4', 'DunedinPACE.5'])

In [22]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=173, out_features=1, bias=True)
)

In [54]:
preprocessing_helper = {
    'gold_standard_probes': PACE_list['DunedinPACE.5'],
    'gold_standard_means': PACE_list['DunedinPACE.5'],
}

In [55]:
weights_dict = {
    'preprocessing': "quantile_normalization_with_gold_standard", 
    'preprocessing_helper': preprocessing_helper,
    'postprocessing': None,
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
}

metadata_dict = {
    'species': 'Homo sapiens',
    'data_type': 'methylation',
    'year': 2022,
    'preprocessing': weights_dict['preprocessing'], 
    'postprocessing': weights_dict['postprocessing'], 
    'citation': "Belsky, Daniel W., et al. \"DunedinPACE, a DNA methylation biomarker of the pace of aging.\" Elife 11 (2022): e73420.",
    'doi': "https://doi.org/10.7554/eLife.73420",
    "notes": "The automatic failure if fewer than 80% of the CpG probes are available is not implemented and left to the user's discretion.",
}

In [56]:
torch.save(weights_dict, '../weights/dunedinpace.pt')
torch.save(metadata_dict, '../metadata/dunedinpace.pt')

In [28]:
os.system("rm DunedinPACE.json")

0